# Installing Important Libraries

In [1]:
%pip install --upgrade --quiet --user langchain langchain-community langchainhub langchain_community langchain-openai langchain-core langchain-google-vertexai langchain-chroma

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pypdf --quiet

In [3]:
!python --version

Python 3.10.14


In [25]:
!pip install -U sentence-transformers

In [3]:
!pip install langchain_huggingface

  Using cached langchain_huggingface-0.0.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
Using cached langchain_huggingface-0.0.3-py3-none-any.whl (17 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)


In [29]:
%pip install --upgrade --quiet  langchain sentence_transformers

Note: you may need to restart the kernel to use updated packages.


# Importing Libraries

In [4]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import json
from langchain_huggingface import HuggingFaceEmbeddings

# Loading Pdf and parsing

In [7]:
directory = "/home/jupyter/Ask_victoria_sop/data/all_json"
filename="data_table_summary_21_06_24.json"
file_path = os.path.join(directory, filename)
with open(file_path, 'r') as f:
    data = json.load(f)

In [8]:
len(data)

49

In [9]:
# data

In [10]:
# data[0]["content"]["text"]

In [11]:
" ".join(data[0]["content"]["text"])

'\nmyTalent is the hiring system where SLT and Talent Acquisition members post open positions, screen\napplications, track candidate progress and extend offers.\n• Stores are responsible for hiring all roles below CEM\n• The Talent Acquisition team is responsible for hiring all CEM and above roles\nREFERENCE GUIDE\nRefer to myTalent resource documents in the myTalent & HR Access Store Reference Guide including:\n• Quick Reference Guides (QRGs)\n• Tips & Tricks\n• Frequently Asked Questions (FAQs)\n• Support Information\n• Candidate Experience\n• Store Connect Video: Build and Grow High Performing Teams > Selection and Interviews\n• Virtual Assistant functionality for step-by-step transaction guidance\nNAVIGATION\nLog into HR Access, click My Talent. If prompted, enter your network ID and password and click Go. Reference the\n“Navigation” section in the myTalent Store Reference Guide for additional navigation resources.\nADMINISTRATION\nAdd SLT members who are responsible for reviewing 

In [12]:
count_html_table=0
count_summary=0
for i in data:
    count_html_table+=len(i["content"]["html_table"])
    count_summary+=len(i["content"]["summary"])
    
print("count_html_table : ", count_html_table)
print("count_summary : ", count_summary)
    
    

count_html_table :  37
count_summary :  37


In [13]:
# data[0]["content"]["html_table"]==[]
# data[0]["content"]["summary"]==[]

In [14]:
def create_langchain_documents(data):
    content_docs = []
    summary_html_docs = []
    
    for item in data:
            m = item['metadata']
            m['page_num']=str(m['page_num'])
            m['actual_page']=str(m['actual_page'])
            text = " ".join(item["content"]["text"])
            content_docs.append(Document(page_content=text, metadata=m))
            
            if item['content']['html_table'] and (len(item["content"]["summary"])==len(item["content"]["html_table"])):
                  for i in range(len(item['content']['html_table'])):
                        summary_html_content = item['content']['summary'][i] + " html table : "+ item['content']['html_table'][i]
                        summary_html_docs.append(Document(page_content=summary_html_content, metadata=m))
                        
                        
    return content_docs,summary_html_docs
                    

In [15]:
content_docs,summary_html_docs=create_langchain_documents(data)

In [16]:
len(summary_html_docs)

37

In [17]:
summary_html_docs[0]

Document(page_content='The table compares and contrasts the responsibilities of the new hire and the SLT (Sales Leadership Team) regarding the completion of the I-9 form, a document used for verifying the identity and employment authorization of individuals working in the United States. Section 1 of the I-9 form, focusing on Associate Information and Verification, must be completed by the new hire either before or on their first day of employment. The new hire is responsible for reviewing and verifying the information provided and digitally signing the form. If the new hire needs assistance, a translator can help them complete this section. It is important to note that the SLT should never enter information in Section 1 on behalf of the associate.Section 2 of the I-9 form, focusing on Employer Review and Verification, must be completed by the SLT on the new hire’s first day of employment. The SLT is responsible for reviewing the documents presented by the new hire, ensuring their authe

In [18]:
# import os
# import json

# # Directory containing the JSON files

# directory = "/home/jupyter/Ask_victoria_sop/data/all_json/v2_with_table_summary"

# documents = []

# # List all JSON files in the directory
# for filename in os.listdir(directory):
#     if filename.endswith(".json"):
#         file_path = os.path.join(directory, filename)
#         with open(file_path, 'r') as f:
#             data = json.load(f)

#         for item in data:
#             m = item['metadata']
#             m['page_num']=str(m['page_num'])
#             m['actual_page']=str(m['actual_page'])
#             text = item["content"]
#             documents.append(Document(page_content=text, metadata=m))

# Now, documents contain all processed Document objects from the JSON files in the directory


In [19]:
# m['page_num']

# Splitting Documents

In [20]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1600, chunk_overlap=240)
splits = text_splitter.split_documents(content_docs)
len(splits)

116

In [21]:
splits_content_table=splits + summary_html_docs

In [22]:
# splits_content_table[115:118]

In [23]:
len(splits)
len(splits_content_table)

153

# Loading Text Embedding model

In [24]:
# embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [25]:
# embeddings2 = VertexAIEmbeddings(model_name="text-embedding-004")

In [35]:
# Requires sentence_transformers>=2.7.0

# from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en-v1.5', show_progress= True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

# Index Creation

In [ ]:

persist_directory="/home/jupyter/Ask_victoria_sop/data/chroma_versions/cdb_1_section_table_summary_1600_240_bge-large"


vectorstore = Chroma.from_documents(collection_name="collection1",documents=splits_content_table, embedding=embeddings,persist_directory=persist_directory)

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
vectorstore._collection.count()

In [ ]:
r1=vectorstore.as_retriever()
r1.invoke("WHEN one should  ENTER TERMINATION IN HR ACCESS for  end of temporary employement?")

In [ ]:
r1.invoke("<html>")

In [108]:
# all_data=vectorstore.get(include =['embeddings','documents','metadatas'])

In [95]:


# structured_data = {
#     'ids': all_data['ids'],
#     'embeddings': all_data['embeddings'] ,
#     'metadatas': all_data['metadatas'],
#     'documents': all_data['documents']
# }

In [96]:
# # Step 4: Serialize the data using pickle
# pickle_file_path = '/home/jupyter/Ask_victoria_sop/chromadb_embeddings/chromadb_data.pkl'
# with open(pickle_file_path, 'wb') as file:
#     pickle.dump(structured_data, file)

In [6]:
# import pickle
# pickle_file_path = '/home/jupyter/Ask_victoria_sop/chromadb_embeddings/chromadb_data.pkl'
# with open(pickle_file_path, 'rb') as file:
#     new=pickle.load(file)